# Migrate SQL Server Instance to Azure SQL VM

## Source SQL Instance
The following code is used to specify the source SQL Server instance.  Data and Server objects will be copied from this server to the target SQL Server instance.  The code below will first prompt for the name of the source SQL Server instance.  Secondarily, the code will use a secure credential prompt for the login credentials for the source instance. 

*Note: the notebook currently is setup for SQL Authentication.  Future updates will add support for multiple authentication types.*


In [ ]:
# set the name or IP address of the source SQL Server instance
Add-Type -Assemblyname PresentationFramework
$sourceInfoBox = New-Object AnyBox.AnyBox

$sourceInfoBox.Prompts = New-AnyBoxPrompt -Name "serverName" -Message "Source SQL Instance name or IP address:" -ValidateNotEmpty
$sourceInfoBox.Buttons = New-AnyBoxButton -Name 'submit' -Text "OK" -IsDefault

$sourceInfoBox.Topmost = $true
$sourceInfoBox.WindowStyle = 'None'

$sourceInfoResponse = $sourceInfoBox | Show-AnyBox
$sourceServerName = $sourceInfoResponse['serverName']

# prompt for username and pw for authentication - supports SQL Authentication currently
$sourceCredential = Get-Credential -Message "Enter Source Login Credentials"

# test the connectivity and display instance information
$sourceTest = Test-DbaConnection -SqlInstance $sourceServerName -SqlCredential $sourceCredential
$sourceTest

# sourceConnection is used in the SQL migration cell below
$sourceConnection = Connect-DbaInstance -SqlInstance $sourceServerName -SqlCredential $sourceCredential

## Target SQL Instance
The following code is used to specify the target SQL Server instance.  Data and Server objects will be copied to this server from the source SQL Server instance.  The code below will first prompt for the name of the target SQL Server instance.  Secondarily, the code will use a secure credential prompt for the login credentials for the target instance. 

*Note: the notebook currently is setup for SQL Authentication.  Future updates will add support for multiple authentication types.*


In [ ]:
# set the name or IP address of the source SQL Server instance
Add-Type -Assemblyname PresentationFramework
$targetInfoBox = New-Object AnyBox.AnyBox

$targetInfoBox.Prompts = New-AnyBoxPrompt -Name "serverName" -Message "Target SQL Instance name or IP address:" -ValidateNotEmpty
$targetInfoBox.Buttons = New-AnyBoxButton -Name 'submit' -Text "OK" -IsDefault

$targetInfoBox.Topmost = $true
$targetInfoBox.WindowStyle = 'None'

$targetInfoResponse = $targetInfoBox | Show-AnyBox
$targetServerName = $targetInfoResponse['serverName']

# prompt for username and pw for authentication - supports SQL Authentication currently
$targetCredential = Get-Credential -Message "Enter Target Login Credentials"

## PowerShell Environment 
$targetTest = Test-DbaConnection -SqlInstance $targetServerName -SqlCredential $targetCredential
$targetTest
$targetConnection = Connect-DbaInstance -SqlInstance $targetServerName -SqlCredential $targetCredential

## Login to Microsoft Azure
To configure and provision resources you must log into your Azure account and set the current subscription that is being used for the target SQL Server instance.  The following code will help you connect your account and choose the correct subscription.  When presented with the list of subscriptions, click on the desired subscription and press OK.

In [ ]:
Connect-AzAccount
$migrationSubscription = Get-AzSubscription | Select-Object -Property Name, Id | Out-GridView -PassThru
Set-AzContext -SubscriptionId $migrationSubscription.Id

## Verify No Active Connections

In [ ]:
#TODO - filter connected proceesses for user connections


Get-DbaProcess -SqlInstance $SourceServerName -SqlCredential $sourceLogin | 
Select Host, login, Program

## Temporary Storage for Data Movement

Offline data migration attempts to use backup to URL and restore from URL as the mechanism for moving data from the source instance to the target instance.  This code will check existance of the specified storage account and container to use for data migration. If the resources do not exist they will be created.

In [ ]:
# TODO - add interactive support for these values
$resourceGroup = "sqlmig"
$blobStorageAccount = "tempsqlmigstorage"
$containerName = "backups"
$location = "West US 2"

# Storage Account
$storageAccount = Get-AzStorageAccount -ResourceGroupName $resourceGroup -Name $blobStorageAccount
if ($storageAccount -eq $null)
{
    # specified storage account does not yet exist, attempt to create it
    $storageAccount = New-AzStorageAccount -ResourceGroupName $resourceGroup -Name $blobStorageAccount -Location $location -SkuName Standard_LRS -Kind StorageV2
}
$storageAccount

# Container
$storageContext = $storageAccount.Context
$storageContainer = Get-AzStorageContainer -Name $containerName -Context $storageContext
if ($storageContainer -eq $null)
{
    #specified storage container does not yet exist, attempt to create it
    $storageContainer = New-AzStorageContainer -Name $containerName -Context $storageContext -Permission Container
}
$storageContainer

# Provide source instance with SAS token for blob access
$sourceSAS = (New-AzStorageAccountSASToken -Service Blob -ResourceType Object -Permission "rw" -Context $storageContext).TrimStart('?')
$sourceCred = New-DbaCredential -SqlInstance $sourceConnection -Name "https://$blobStorageAccount.blob.core.windows.net/$containerName" -Identity "SHARED ACCESS SIGNATURE" -SecurePassword (ConvertTo-SecureString $sourceSAS -AsPlainText -Force) -Force
$sourceCred

$targetSAS = (New-AzStorageAccountSASToken -Service Blob -ResourceType Object -Permission "rw" -Context $storageContext).TrimStart('?') # -ResourceType Container,Object
$targetCred = New-DbaCredential -SqlInstance $targetConnection -Name "https://$blobStorageAccount.blob.core.windows.net/$containerName" -Identity "SHARED ACCESS SIGNATURE" -SecurePassword (ConvertTo-SecureString $targetSAS -AsPlainText -Force) -Force
$targetCred



## Start the Server Migration

The follow code executes the migration of objects from the source SQL Server instance to the target SQL Server instance.  There are a couple of options of interest:

|Parameter|Description|
|---|---|
|Verbose|Include detailed logging information during the migration process|
|WhatIf|Display what the migration process is going to do but do **not** actually perform the migration|

The above parameters can be turned off by changing the $true value following the parameter to $false in the code cell below.  

In [ ]:
$migrationParams = @{
    Verbose = $true
    WhatIf = $true
    Source = $sourceConnection
    Destination = $targetConnection
    BackupRestore = $true
    SharedPath = "https://$blobStorageAccount.blob.core.windows.net/$containerName" 
}

Start-DbaMigration @migrationParams